In [ ]:
#INFO - Librarite e perdorura 

import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# INFO - Leximi i dataset-it nga source!

dataset=pd.read_csv('../data/gjobat-e-leshuara.csv')

In [ ]:
# Para Procesimi

# Kontrollojme se a mungon ndonje vlere
dataset.isnull().sum()

# Ndryshimi i emrave te kolonave te caktuara
dataset.rename(columns={'LLOJI_KOMPAN': 'LLOJI_KOMPANISE', 'NR_TATIM': 'NR_TATIMIT'}, inplace=True)
print(dataset)

In [ ]:
# Definimi i tipeve

print(dataset.dtypes)

In [ ]:
# Paraqitja e numrit te rreshtave dhe kolonave

dataset.shape

In [ ]:
# Integrimi

dataset['MUAJI/VITI'] = dataset['MUAJI'].astype(str) + '/' + dataset['VITI'].astype(str)

In [ ]:
# Agregimi

Vlerat_bazuar_ne_vite = dataset.groupby('VITI')['VLERA'].sum()
print(Vlerat_bazuar_ne_vite)

In [ ]:
# Mostrimi

gjatesia_e_mostres = 100 
mostra = dataset.sample(n=gjatesia_e_mostres, random_state=42)

print("Mostrimi random:")
print(mostra)

In [ ]:
# Pastrimi

print("Numri i rreshtave dhe kolonave para pastrimit")
print(dataset.shape)

# Emertimi i rregullt i kolones "KOMUNA"
dataset['KOMUNA'] = dataset['KOMUNA'].str.replace('Ã‹', 'Ë')

print("Paraqitja e kolonave qe permbajne vlera null:")
print(dataset.isnull().sum())
print("\n")

print("Rreshtat duplikat:")
print(dataset.duplicated().sum())
print("\n")

#Largon kolonen e pare
dataset = dataset.drop(dataset.columns[0], axis=1)

# Outliners
# Llogarit z-scores për kolonën e përcaktuar
z_scores = (dataset['VLERA'] - dataset['VLERA'].mean()) / dataset['VLERA'].std()

# Caktimi i pragut per konsiderimin e te dhënave si outliers (p.sh., 3 devijimet standard)
pragu = 3

# Krijo një maske per te identifikuar outliers
maska_outliers = abs(z_scores) >= pragu

# Nxirr dhe printo vlerat që u larguan (outliers)
vlerat_e_larguara = dataset.loc[maska_outliers, 'VLERA']
print("Vlerat që u larguan (outliers):")
print(vlerat_e_larguara)

# Krijo një dataset të ri pa outliers
dataset = dataset[~maska_outliers]

# Shfaq informata rreth datasetit para dhe pas largimit të outliers
print(f"Forma origjinale e datasetit: {dataset.shape}")

# Zevendsimi i vlerave lowercase ne uppercase per kolonen 'KOMUNA'
dataset['KOMUNA'] = dataset['KOMUNA'].str.upper()

print("Numri i rreshtave dhe kolonave pas pastrimit")
print(dataset.shape)


In [ ]:
# Identifikimi

# Shtoni identifikuesin në kolonën e parë
dataset.insert(0, 'ID_UNIKE', dataset['MUAJI'].astype(str)+'/'+dataset['VITI'].astype(str) + '_' + dataset['PERSHKRIMI_SEKTORIT'] + '_' + dataset['LLOJI_KOMPANISE'])

print("Printimi i dataset-it pas insertimit te kolones unike.")
print(dataset)

In [ ]:
#Binarizimi
average_vlera = dataset['VLERA'].mean()

dataset['BINARIZED_VLERA'] = dataset['VLERA'].apply(lambda x: 1 if x > average_vlera else 0)

print(f"Vlera mesatare eshte: {average_vlera}")

print(dataset)